In [2]:
%cd /bilinear-feature-circuits

/bilinear-feature-circuits


In [3]:
import os
import sys
parent_dir = os.path.abspath('.')
sys.path.append(parent_dir)
from model_utils import *
t.set_printoptions(precision=4, sci_mode=False)

Clean and patch inputs of different shapes.
Clean: 3 Patch: 4
Clean and patch inputs of different shapes.
Clean: 4 Patch: 3

Integrated Gradient estimation



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



Initial trace


Patching part

Error creating visualizer for Embedding(32000, 1024): File does not contain tensor 0-resid_pre


In [4]:
print_effects_and_residual_effects(attns[0], False)

Positive effects at Attention(
  (rotary): Rotary()
  (qkv): Linear(in_features=1024, out_features=3072, bias=True)
  (o): Linear(in_features=1024, out_features=1024, bias=False)
  (softmax): Softmax(dim=-1)
):
torch.return_types.topk(
values=tensor([[[    1.4124,     0.4015,     0.0221,     0.0081,     0.0072,
              0.0044,     0.0032,     0.0029,     0.0026,     0.0015,
              0.0013,     0.0011,     0.0011,     0.0010,     0.0007,
              0.0003,     0.0002,     0.0002,     0.0001,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000],
         [    0.0105,     0.0052,     0.0038,     0.0030,     0.0026,
              0.0024,     0.0015,     0.0012,     0.0011,     0.0009,
              0.0008,     0.0008,     0.0007,     0.0003,     0.0001,
              0.0001,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000,     0.0000, 

In [4]:
attn0_out = compute_aggregated_latents(attns[0], {1: [3514,103], 2: [3514,2764], 3: [3514,1703], 4: [3514, 5042]}, norm_method = None)

In [5]:
#replace attention:
with model.trace(short_input):
    model._envoy.transformer.h[0].attn.output[0,1:] = attn0_out[0]
    attn0_replaced = model.output
    new_mlp_latents = dictionaries[mlps[0]].encode(model._envoy.transformer.h[0].mlp.output)
    new_mlp_latents.save()
    attn0_replaced.save()

compare_distributions(attn0_replaced, baseline_out, 298)

Correct Token:
  Token ID: 298, Token: to
  Reference Log Prob: -0.1197
  New Log Prob:       -0.2632
  Change:             -0.1435

Reference Distribution Top 5 Tokens:
  Rank 1: ID 298, Token: to, Log Prob: -0.1197
  Rank 2: ID 354, Token: for, Log Prob: -3.6161
  Rank 3: ID 297, Token: in, Log Prob: -4.4850
  Rank 4: ID 390, Token: as, Log Prob: -4.5395
  Rank 5: ID 579, Token: so, Log Prob: -4.6242

New Distribution Top 5 Tokens:
  Rank 1: ID 298, Token: to, Log Prob: -0.2632
  Rank 2: ID 297, Token: in, Log Prob: -3.3576
  Rank 3: ID 354, Token: for, Log Prob: -3.5114
  Rank 4: ID 579, Token: so, Log Prob: -3.7230
  Rank 5: ID 369, Token: that, Log Prob: -3.8358

Overlap in Top 5 Tokens:
  in, to, for, so


In [6]:
print(t.topk(new_mlp_latents[0,1:], k = 31))

torch.return_types.topk(
values=tensor([[4.2448, 3.2571, 3.0683, 2.9286, 2.8218, 2.7125, 2.6674, 2.4292, 2.3732,
         2.2394, 2.0772, 1.9808, 1.8038, 1.7446, 1.6532, 1.4899, 1.3204, 1.3106,
         1.2189, 1.1216, 0.9584, 0.9271, 0.8843, 0.8699, 0.5678, 0.5538, 0.4690,
         0.3779, 0.3720, 0.3298, 0.0000],
        [4.3517, 4.3262, 4.1144, 3.6906, 3.6122, 3.5934, 3.4363, 3.3969, 3.3551,
         3.0004, 2.7879, 2.5060, 2.0691, 2.0239, 1.9917, 1.9281, 1.8964, 1.8300,
         1.7971, 1.7694, 1.6457, 1.3221, 1.0970, 1.0534, 1.0524, 0.9772, 0.9130,
         0.6048, 0.4634, 0.4530, 0.0000],
        [2.4135, 2.3403, 2.2945, 2.1932, 2.0827, 1.8783, 1.8047, 1.6589, 1.3712,
         1.3683, 1.2982, 1.0325, 1.0047, 0.9931, 0.9724, 0.9546, 0.9190, 0.8521,
         0.8505, 0.8238, 0.7539, 0.7000, 0.6755, 0.6502, 0.6229, 0.6176, 0.5922,
         0.5207, 0.5025, 0.4818, 0.0000],
        [6.9547, 4.4225, 4.2674, 3.6945, 3.6849, 3.3687, 3.2781, 3.2281, 3.0975,
         2.9361, 2.7653, 2.4209,

In [7]:
model._model.w_e.shape

torch.Size([1024, 32000])

In [8]:
#weird thing with the MLP layer
print_effects_and_residual_effects(mlps[0], False)

Positive effects at MLP(
  (w): Bilinear(
    in_features=1024, out_features=8192, bias=True
    (gate): Identity()
  )
  (p): Linear(in_features=4096, out_features=1024, bias=True)
):
torch.return_types.topk(
values=tensor([[[    0.0180,     0.0119,     0.0117,     0.0087,     0.0072,
              0.0055,     0.0053,     0.0047,     0.0041,     0.0041,
              0.0035,     0.0003,     0.0001,     0.0000,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000],
         [    0.0456,     0.0345,     0.0297,     0.0121,     0.0076,
              0.0055,     0.0041,     0.0037,     0.0029,     0.0016,
              0.0014,     0.0008,     0.0007,     0.0006,     0.0002,
              0.0000,     0.0000,     0.0000,     0.0000,     0.0000,
              0.0000,     0.0000,     0.0000,     0.0000,   

In [9]:
from sae import Tracer
mlp_0_tracer = Tracer(model._model, 0, out = dict(name = 'mlp-out', expansion = 8, k = 30), inp = dict(name = 'resid-mid', expansion = 8, k = 30))
mlp_0_inp_vis = Visualizer(model._model, mlp_0_tracer.inp)
mlp_0_out_vis = Visualizer(model._model, mlp_0_tracer.out)

In [17]:
t.stack((attn0_out[0][-1]/attn0_out[0][-1].norm(),DESIGNED/DESIGNED.norm()), dim = 1)

tensor([[-0.0322,  0.0873],
        [-0.0208,  0.0505],
        [-0.0175,  0.0118],
        ...,
        [ 0.0096,  0.0349],
        [ 0.0021,  0.0787],
        [-0.0465, -0.0740]], device='cuda:0', grad_fn=<StackBackward0>)

In [18]:
mlp_0_tracer.custom_q(mlp_0_tracer.out.w_dec.weight.data[:, 8184],t.stack((attn0_out[0][-1]/attn0_out[0][-1].norm(),DESIGNED/DESIGNED.norm()), dim = 1))

tensor([[ 0.0039, -0.0043],
        [-0.0043,  0.0154]], device='cuda:0', grad_fn=<ViewBackward0>)

In [24]:
with model.trace(short_input):
    mlp_0_in = model._envoy.transformer.h[0].n2.output
    mlp_0_in.save()
    mlp_0_out = model._envoy.transformer.h[0].mlp.output
    mlp_0_out.save()
q_8184 = mlp_0_tracer.custom_q(mlp_0_tracer.out.w_dec.weight.data[:, 8184])

In [38]:
mlp_0_tracer.out.w_dec.weight.data[:, 8184]

tensor([0.0080, 0.0159, 0.0670,  ..., 0.0510, 0.0329, 0.0048], device='cuda:0')

In [39]:
mlp_0_tracer.out_latents[8184]

tensor([ 0.0183,  0.0201,  0.0076,  ...,  0.0574, -0.0068, -0.0547],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [25]:
q_8184_2 = mlp_0_tracer.q(8184)

In [36]:
mlp_0_in[:,-1] @ q_81842 @ mlp_0_in[:,-1].T

tensor([[8.3424]], device='cuda:0', grad_fn=<MmBackward0>)

In [34]:
mlp_0_out[:,-1] @ mlp_0_tracer.out.w_dec.weight.data[:, 8184].unsqueeze(1)

tensor([[1.9216]], device='cuda:0', grad_fn=<MmBackward0>)

In [5]:
feat_57_acts = compute_aggregated_latents(attns[0], {4: [57]}, norm_method = None)

In [9]:
feat_57_acts[0].shape

torch.Size([4, 1024])

In [10]:
longer_input = input_tensor[:, :6]
baseline_longer = model.forward(longer_input)
with model.trace(longer_input):
    model._envoy.transformer.h[0].attn.output[0,-2] -= feat_57_acts[0][-1]
    ablated_longer = model.output.save()


In [11]:
compare_distributions(ablated_longer, baseline_longer, 27854)

Correct Token:
  Token ID: 27854, Token: activate
  Reference Log Prob: -3.4814
  New Log Prob:       -3.4586
  Change:             0.0227

Reference Distribution Top 5 Tokens:
  Rank 1: ID 8366, Token: trigger, Log Prob: -2.2230
  Rank 2: ID 347, Token: be, Log Prob: -2.5693
  Rank 3: ID 4244, Token: cause, Log Prob: -2.6290
  Rank 4: ID 27854, Token: activate, Log Prob: -3.4814
  Rank 5: ID 1840, Token: keep, Log Prob: -3.6992

New Distribution Top 5 Tokens:
  Rank 1: ID 8366, Token: trigger, Log Prob: -2.2237
  Rank 2: ID 347, Token: be, Log Prob: -2.5647
  Rank 3: ID 4244, Token: cause, Log Prob: -2.6401
  Rank 4: ID 27854, Token: activate, Log Prob: -3.4586
  Rank 5: ID 1840, Token: keep, Log Prob: -3.6425

Overlap in Top 5 Tokens:
  be, keep, cause, trigger, activate
